# import data + preprocess data with str_to_label

In [9]:
import pandas as pd
train_df = pd.read_csv('/ml_data/train.csv').drop(columns = ["Unnamed: 0"])
dev_df = pd.read_csv('/ml_data/dev.csv').drop(columns = ["Unnamed: 0"])
test_df = pd.read_csv('/ml_data/test.csv').drop(columns = ["Unnamed: 0"])
labels = list(set(train_df['label'].to_list()))
label_to_idx = {label:i for i,label in enumerate(labels)}
train_df['label'] = train_df['label'].apply(lambda x: label_to_idx[x])
dev_df['label'] = dev_df['label'].apply(lambda x: label_to_idx[x])
test_df['label'] = test_df['label'].apply(lambda x: label_to_idx[x])

label_list = [0] * len(label_to_idx)
for label in label_to_idx:
    label_list[label_to_idx[label]] = label 

## ที่ทำ label_to_idx เพื่อจะได้เช็คได้ว่ามันเรียงเลขสำหรับ text classification ยังไง

In [11]:
label_to_idx

{'neu': 0, 'pos': 1, 'neg': 2}

In [12]:
#read file from google drive
from script.preprocess import str_to_label
import pandas as pd
# train_df = str_to_label(pd.read_csv('/ml_data/train.csv').drop(columns = ["Unnamed: 0"]), "label")
# dev_df = str_to_label(pd.read_csv('/ml_data/dev.csv').drop(columns = ["Unnamed: 0"]), "label")
# test_df = str_to_label(pd.read_csv('/ml_data/test.csv').drop(columns = ["Unnamed: 0"]), "label")
train_df = pd.read_csv('/ml_data/train.csv').drop(columns = ["Unnamed: 0"])
dev_df = pd.read_csv('/ml_data/dev.csv').drop(columns = ["Unnamed: 0"])
test_df = pd.read_csv('/ml_data/test.csv').drop(columns = ["Unnamed: 0"])

# transform data

In [13]:
from script.transform import transform
test_dataset = transform(train_df, dev_df, test_df, "label")

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [5]:
test_dataset['train']['labels']

tensor([0, 1, 0,  ..., 2, 2, 0])

# train 
(ขออนุญาติไม่รัน มันนานขี้เกียจรอ)

In [7]:
from transformers import TrainingArguments
from script.training import train_model
model_name = 'airesearch/wangchanberta-base-att-spm-uncased'
args2 = TrainingArguments(
    output_dir='netflix-result',
    learning_rate=1e-4,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    evaluation_strategy='steps', 
    eval_steps=10,
    #save_strategy='epochs'    
)
unique_values = len(train_df["label"].unique())
train_model(model_name, unique_values, test_dataset, args2)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--airesearch--wangchanberta-base-att-spm-uncased/snapshots/abe46f39cf2c911a6ad5ec8299bdf7503edc95e4/config.json
Model config CamembertConfig {
  "_name_or_path": "airesearch/wangchanberta-base-att-spm-uncased",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

# load_model and predict

In [14]:
from script.training import model, _tokenizer
import torch
unique_values = len(train_df["label"].unique())
model_ja = model("/model_artifact", unique_values)
tokenizer = _tokenizer("/model_artifact")
dict_ja = {'pos': 0, 'neu': 1, 'neg': 2}
def get_keys(val, my_dict):
    for key, value in my_dict.items():
        if val == value:
            return key
        
def predict_unseen(unseen_text):
    input_ids = tokenizer.encode(unseen_text, add_special_tokens=True)
    input_ids = torch.tensor(input_ids).unsqueeze(0)
    labels = torch.tensor([1]).unsqueeze(0)
    outputs = model_ja(input_ids, labels=labels)
    logits = outputs[0]
    return get_keys(logits.argmax().item(), dict_ja)

In [15]:
unseen_text = "ไม่ชอบหนังเรื่องนี้เลยหว่ะ"
predict_unseen(unseen_text)

'pos'

In [17]:
from script.predict_from_pretrain import predict_unseen
from script.training import model, _tokenizer
import torch
unique_values = len(train_df["label"].unique())
model_ja = model("/model_artifact", unique_values)
tokenizer = _tokenizer("/model_artifact")
unseen_text = "ไม่ชอบหนังเรื่องนี้เลยหว่ะ"
predict_unseen(unseen_text, model_ja, tokenizer)

'pos'